In [142]:
import pandas as pd
import glob
import os

In [143]:
notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\raw\\*.csv"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string

In [144]:
csv_files = glob.glob(
    modified_path
)


# Loading and Combining all month data

In [145]:
dfs = []

# Read each CSV file into a DataFrame and add it to the list
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame vertically
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

# sort by Date/Time and rest index
combined_df.sort_values(by="Date/Time", inplace=True)
combined_df.reset_index(inplace=True)
combined_df.drop(columns='index', inplace=True)

# transform to pd_date
combined_df["Date"] = pd.to_datetime(combined_df["Date/Time"])
combined_df["Date_1"] = combined_df["Date"].dt.date
combined_df["Hour"] = combined_df["Date"].dt.hour

combined_df.head()

,Date/Time,Lat,Lon,Base,Date,Date_1,Hour
0,4/1/2014 0:00:00,40.7637,-73.9600,B02598,2014-04-01 00:00:00,2014-04-01,0
1,4/1/2014 0:00:00,40.7188,-73.9863,B02598,2014-04-01 00:00:00,2014-04-01,0
2,4/1/2014 0:00:00,40.7215,-73.9952,B02682,2014-04-01 00:00:00,2014-04-01,0
3,4/1/2014 0:01:00,40.7355,-73.9966,B02617,2014-04-01 00:01:00,2014-04-01,0
4,4/1/2014 0:02:00,40.7184,-73.9601,B02682,2014-04-01 00:02:00,2014-04-01,0


In [146]:
file_name = "combined_data_all_month.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string + file_name 
combined_df.to_csv(modified_path, index=True)

# Loading and preparing the april data

In [147]:
file_name = "data-apr14.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\raw\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]


modified_path += "\\" + replacement_string + file_name 

In [148]:
df = pd.read_csv(modified_path)

In [149]:
df["Date"] = pd.to_datetime(df["Date/Time"])
df["index"] = df.index
df["Date_1"] = df["Date"].dt.date
df["Hour"] = df["Date"].dt.hour


df_grouped_april = (
    df.groupby(["Date_1", "Hour"], as_index=False)
    .agg(
        {
            "index": "count",  # Replace 'column1' with the name of the column you want to sum
        }
    )
    .reset_index()
)

# Convert the 'Date' column to datetime format
df_grouped_april.drop(columns="level_0", inplace=True)
df_grouped_april.columns = ["Date", "Hour", "Count"]
df_grouped_april["Date"] = pd.to_datetime(df_grouped_april["Date"])

df_grouped_april["CombinedDateTime"] = df_grouped_april["Date"] + pd.to_timedelta(
    df_grouped_april["Hour"], unit="h"
)
df_grouped_april.set_index("CombinedDateTime", inplace=True)
df_grouped_april.index.name = None

In [150]:
file_name = "april_data_processed.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string + file_name 

# Save the DataFrame to the CSV file in the specified folder
df_grouped_april.to_csv(modified_path, index=True)

## April Data sliced into weekdays and weekend

In [151]:
# Creating weekdays for improving the
df_grouped_april["Weekday"] = df_grouped_april["Date"].dt.weekday

weekday_names = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
df_grouped_april["Weekday"] = df_grouped_april["Weekday"].map(lambda x: weekday_names[x])


# lets split up the date and see if we can achiev better results.

weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
weekend = ["Saturday", "Sunday"]

# Filter the DataFrame
df_april_week = df_grouped_april.query("Weekday in @weekdays")
df_april_end = df_grouped_april.query("Weekday in @weekend")

# Differencing
df_april_week["diff_Count1"] = df_april_week["Count"].diff(periods=1)
df_april_week["diff_Count24+1"] = df_april_week["diff_Count1"].diff(periods=24)
df_april_week["diff_Count24"] = df_april_week["Count"].diff(periods=24)

df_april_end["diff_Count1"] = df_april_end["Count"].diff(periods=1)
df_april_end["diff_Count24+1"] = df_april_end["diff_Count1"].diff(periods=24)
df_april_end["diff_Count24"] = df_april_end["Count"].diff(periods=24)


# Save the DataFrame to the CSV file in the specified folder
file_name = "df_april_week.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]
modified_path += "\\" + replacement_string + file_name 

df_april_week.to_csv(modified_path, index=True)

# Save the DataFrame to the CSV file in the specified folder
file_name = "df_april_end.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]
modified_path += "\\" + replacement_string + file_name 

df_april_end.to_csv(modified_path, index=True)

C:\Users\Max_G\AppData\Local\Temp\ipykernel_24716\1331811347.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_april_week["diff_Count1"] = df_april_week["Count"].diff(periods=1)
C:\Users\Max_G\AppData\Local\Temp\ipykernel_24716\1331811347.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_april_week["diff_Count24+1"] = df_april_week["diff_Count1"].diff(periods=24)
C:\Users\Max_G\AppData\Local\Temp\ipykernel_24716\1331811347.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

# Loading and preparing the may data


In [152]:
file_name = "data-may14.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\raw\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]


modified_path += "\\" + replacement_string + file_name 
df_may = pd.read_csv(modified_path)

In [153]:
df_may = pd.read_csv(
    r"C:\Users\Max_G\OneDrive\IUBH\5_Semester\Model_Engeneering\Public_Transport_Forecasting\data\raw\data-may14.csv"
)

df_may["Date"] = pd.to_datetime(df_may["Date/Time"])
df_may["index"] = df_may.index
df_may["Date_1"] = df_may["Date"].dt.date
df_may["Hour"] = df_may["Date"].dt.hour

df_grouped_may = (
    df_may.groupby(["Date_1", "Hour"], as_index=False)
    .agg(
        {
            "index": "count",  # Replace 'column1' with the name of the column you want to sum
        }
    )
    .reset_index()
)


# Convert the 'Date' column to datetime format
df_grouped_may.drop(columns="level_0", inplace=True)
df_grouped_may.columns = ["Date", "Hour", "Count"]
df_grouped_may["Date"] = pd.to_datetime(df_grouped_may["Date"])

df_grouped_may["CombinedDateTime"] = df_grouped_may["Date"] + pd.to_timedelta(
    df_grouped_may["Hour"], unit="h"
)
df_grouped_may.set_index("CombinedDateTime", inplace=True)
df_grouped_may.index.name = None

In [154]:
file_name = "may_data_processed.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string + file_name 

# Save the DataFrame to the CSV file in the specified folder
df_grouped_may.to_csv(modified_path, index=True)

## May Data sliced into weekdays and weekend

In [155]:
# Creating weekdays for improving the
df_grouped_may["Weekday"] = df_grouped_may["Date"].dt.weekday

weekday_names = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
df_grouped_may["Weekday"] = df_grouped_may["Weekday"].map(lambda x: weekday_names[x])


# lets split up the date and see if we can achiev better results.

weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
weekend = ["Saturday", "Sunday"]

# Filter the DataFrame
df_may_week = df_grouped_may.query("Weekday in @weekdays")
df_may_end = df_grouped_may.query("Weekday in @weekend")



# Save the DataFrame to the CSV file in the specified folder
file_name = "df_may_week.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]
modified_path += "\\" + replacement_string + file_name 

df_may_week.to_csv(modified_path, index=True)


# Save the DataFrame to the CSV file in the specified folder
file_name = "df_may_end.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]
modified_path += "\\" + replacement_string + file_name 

df_may_end.to_csv(modified_path, index=True)

# Loading and Preparing the clustered data

In [156]:
file_name = "combined_month_clustered_6.csv"

notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\interim\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]


modified_path += "\\" + replacement_string + file_name 

df = pd.read_csv(modified_path, index_col=[0])
df.Date = pd.to_datetime(df['Date'])
df["index"] = df.index

In [157]:
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekend = [ 'Saturday', 'Sunday']

## Splitting in cluster, month, weekday/week

In [158]:
df_all_data_grouped = (
            df.groupby(["Date_1", "Hour"], as_index=False)
            .agg({"index": "count"})
            .reset_index()
        )

df_all_data_grouped.drop(columns="level_0", inplace=True)
df_all_data_grouped.columns = ["Date", "Hour", "Count"]
df_all_data_grouped["Date"] = pd.to_datetime(df_all_data_grouped["Date"])
df_all_data_grouped["CombinedDateTime"] = (
            df_all_data_grouped["Date"]
            + pd.to_timedelta(df_all_data_grouped["Hour"], unit="h")
        )
df_all_data_grouped.set_index("CombinedDateTime", inplace=True)
df_all_data_grouped.index.name = None

In [159]:
df_all_data_grouped["Weekday"] = df_all_data_grouped["Date"].dt.weekday
df_all_data_grouped["Weekday"] = df_all_data_grouped["Weekday"].map(lambda x: weekday_names[x])

# Split the data into weekday and weekend DataFrames
df_all_data_grouped_week = df_all_data_grouped.query("Weekday in @weekdays")
# current_df_week.reset_index(drop=True, inplace=True)
df_all_data_grouped_end = df_all_data_grouped.query("Weekday in @weekend")
# current_df_end.reset_index(drop=True, inplace=True)

 # Define the file names based on the label and month
weekday_file_name = f"df_all_data_grouped_weekday.csv"
weekend_file_name = f"df_all_data_grouped_weekend.csv"
grouped_file_name = f"df_all_data_grouped.csv"


notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\interim\\Clustered Data\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path_weekday = modified_path + "\\" + replacement_string + weekday_file_name
modified_path_weekend = modified_path + "\\" + replacement_string + weekend_file_name
modified_path_grouped = modified_path + "\\" + replacement_string + grouped_file_name 


# Save the DataFrames to CSV files
df_all_data_grouped_end.to_csv(modified_path_weekend, index=True)
df_all_data_grouped_week.to_csv(modified_path_weekday, index=True)
df_all_data_grouped.to_csv(modified_path_grouped, index=True)

In [160]:
label_list = df['labels'].unique()

# Create a dictionary to store data frames for each label and month
neighborhood_dict = {}

# Define the list of months
month_list = [4, 5, 6, 7, 8, 9]

# Iterate through labels and months
for label in label_list:
    neighborhood_dict[label] = {}  
    for month in month_list:
        # Filter the DataFrame for the current label and month
        df_filtered = df[(df['labels'] == label) & (df['Date'].dt.month == month)]
        neighborhood_dict[label][month] = df_filtered

In [161]:
nested_dict = {}

for label in neighborhood_dict:
    nested_dict[label] = {}  # Create an inner dictionary for each label
    for month in neighborhood_dict[label]:
        # Extract the DataFrame for the current label and month
        current_df = neighborhood_dict[label][month]
        
        # Perform your DataFrame manipulations here
        current_df_grouped = (
            current_df.groupby(["Date_1", "Hour"], as_index=False)
            .agg({"index": "count"})
            .reset_index()
        )
        current_df_grouped.drop(columns="level_0", inplace=True)
        current_df_grouped.columns = ["Date", "Hour", "Count"]
        current_df_grouped["Date"] = pd.to_datetime(current_df_grouped["Date"])
        current_df_grouped["CombinedDateTime"] = (
            current_df_grouped["Date"]
            + pd.to_timedelta(current_df_grouped["Hour"], unit="h")
        )
        current_df_grouped.set_index("CombinedDateTime", inplace=True)
        current_df_grouped.index.name = None
        
        # Add the current DataFrame to the inner dictionary
        nested_dict[label][month] = current_df_grouped

In [162]:
transformed_nested_dict = {}

# Loop through the original nested_dict
for label in nested_dict:
    transformed_nested_dict[label] = {}  # Create an inner dictionary for each label
    for month in nested_dict[label]:
        # Extract the DataFrame for the current label, month, and hour
        current_df = nested_dict[label][month]

        # Apply the transformations
        current_df["Weekday"] = current_df["Date"].dt.weekday
        current_df["Weekday"] = current_df["Weekday"].map(lambda x: weekday_names[x])

        # Split the data into weekday and weekend DataFrames
        current_df_week = current_df.query("Weekday in @weekdays")
        # current_df_week.reset_index(drop=True, inplace=True)
        current_df_end = current_df.query("Weekday in @weekend")
        # current_df_end.reset_index(drop=True, inplace=True)

        # Store the transformed DataFrames in the inner dictionary
        transformed_nested_dict[label][month] = {
            "Weekday": current_df_week,
            "Weekend": current_df_end,
        }

In [163]:
notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\interim\\Clustered Data\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string

In [164]:
for label in transformed_nested_dict:
    for month in transformed_nested_dict[label]:
        # Extract the transformed DataFrames for weekday and weekend
        weekday_df = transformed_nested_dict[label][month]["Weekday"]
        weekend_df = transformed_nested_dict[label][month]["Weekend"]

        # Define the file names based on the label and month
        weekday_file_name = f"df_{label}_month_{month}_weekday.csv"
        weekend_file_name = f"df_{label}_month_{month}_weekend.csv"

        # Create the full paths for saving
        weekday_full_path = modified_path + weekday_file_name
        weekend_full_path = modified_path + weekend_file_name

        # Save the DataFrames to CSV files
        weekday_df.to_csv(weekday_full_path, index=True)
        weekend_df.to_csv(weekend_full_path, index=True)

### Splitting in Cluster and weekday/week

In [165]:
label_list = df['labels'].unique()

# Create a dictionary to store data frames for each label and month
neighborhood_dict = {}

# Iterate through labels and months
for label in label_list:
    # Filter the DataFrame for the current label and month
    df_filtered = df[(df['labels'] == label)]
    neighborhood_dict[label]= df_filtered


In [166]:
neighborhood_dict_grouped = {}

for label in neighborhood_dict:
    
    current_df = neighborhood_dict[label]
        
        # Perform your DataFrame manipulations here
    current_df_grouped = (
        current_df.groupby(["Date_1", "Hour"], as_index=False)
        .agg({"index": "count"})
        .reset_index()
    )
    current_df_grouped.drop(columns="level_0", inplace=True)
    current_df_grouped.columns = ["Date", "Hour", "Count"]
    current_df_grouped["Date"] = pd.to_datetime(current_df_grouped["Date"])
    current_df_grouped["CombinedDateTime"] = (
        current_df_grouped["Date"]
        + pd.to_timedelta(current_df_grouped["Hour"], unit="h")
    )
    current_df_grouped.set_index("CombinedDateTime", inplace=True)
    current_df_grouped.index.name = None

    
    # Add the current DataFrame to the inner dictionary
    neighborhood_dict_grouped[label] = current_df_grouped

In [167]:
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekend = [ 'Saturday', 'Sunday']

In [168]:
# there are missing values after aggregating, need to fill in 0.
for label in neighborhood_dict_grouped:
    
    current_df_grouped = neighborhood_dict_grouped[label]

    start_date = neighborhood_dict_grouped[0].index[0] 
    end_date = neighborhood_dict_grouped[0].index[-1] 
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    current_df_grouped = current_df_grouped.reindex(date_range)
    current_df_grouped['Count'].fillna(0, inplace=True)
    current_df_grouped['Date'] = current_df_grouped.index.date
    current_df_grouped['Hour'] = current_df_grouped.index.hour

    neighborhood_dict_grouped[label] = current_df_grouped


In [169]:
neighborhood_dict_grouped_diff_week = {}

# Loop through the original nested_dict
for label in neighborhood_dict_grouped:
    # Extract the DataFrame for the current label, month, and hour
    current_df = neighborhood_dict_grouped[label]
    
    current_df['Date'] = pd.to_datetime(current_df_grouped["Date"])
        # Apply the transformations
    current_df["Weekday"] = current_df["Date"].dt.weekday
    current_df["Weekday"] = current_df["Weekday"].map(lambda x: weekday_names[x])

    # Split the data into weekday and weekend DataFrames
    current_df_week = current_df.query("Weekday in @weekdays")
    current_df_end = current_df.query("Weekday in @weekend")
    

    # Store the transformed DataFrames in the inner dictionary
    neighborhood_dict_grouped_diff_week[label] = {
        "Weekday": current_df_week,
        "Weekend": current_df_end,
    }

In [170]:
notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)


replacement_string = "data\\interim\\Cluster_Data_All_Month\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string

In [171]:
for label in  neighborhood_dict_grouped_diff_week:
    # Extract the transformed DataFrames for weekday and weekend
    weekday_df = neighborhood_dict_grouped_diff_week[label]["Weekday"]
    weekend_df = neighborhood_dict_grouped_diff_week[label]["Weekend"]

    # Define the file names based on the label and month
    weekday_file_name = f"df_{label}_weekday.csv"
    weekend_file_name = f"df_{label}_weekend.csv"

    # Create the full paths for saving
    weekday_full_path = modified_path + weekday_file_name
    weekend_full_path = modified_path + weekend_file_name

    # Save the DataFrames to CSV files
    weekday_df.to_csv(weekday_full_path, index=True)
    weekend_df.to_csv(weekend_full_path, index=True)

# Splitting the data in clusters and then grouping

In [172]:
label_list = df['labels'].unique()

# Create a dictionary to store data frames for each label and month
neighborhood_dict = {}

# Iterate through labels and months
for label in label_list:
    # Filter the DataFrame for the current label and month
    df_filtered = df[(df['labels'] == label)]
    neighborhood_dict[label]= df_filtered


In [173]:
neighborhood_dict_grouped = {}

for label in neighborhood_dict:
    
    current_df = neighborhood_dict[label]
        
        # Perform your DataFrame manipulations here
    current_df_grouped = (
        current_df.groupby(["Date_1", "Hour"], as_index=False)
        .agg({"index": "count"})
        .reset_index()
    )
    current_df_grouped.drop(columns="level_0", inplace=True)
    current_df_grouped.columns = ["Date", "Hour", "Count"]
    current_df_grouped["Date"] = pd.to_datetime(current_df_grouped["Date"])
    current_df_grouped["CombinedDateTime"] = (
        current_df_grouped["Date"]
        + pd.to_timedelta(current_df_grouped["Hour"], unit="h")
    )
    current_df_grouped.set_index("CombinedDateTime", inplace=True)
    current_df_grouped.index.name = None

    
    # Add the current DataFrame to the inner dictionary
    neighborhood_dict_grouped[label] = current_df_grouped

In [174]:
# there are missing values after aggregating, need to fill in 0.
for label in neighborhood_dict_grouped:
    
    current_df_grouped = neighborhood_dict_grouped[label]

    start_date = neighborhood_dict_grouped[0].index[0] 
    end_date = neighborhood_dict_grouped[0].index[-1] 
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    current_df_grouped = current_df_grouped.reindex(date_range)
    current_df_grouped['Count'].fillna(0, inplace=True)
    current_df_grouped['Date'] = current_df_grouped.index.date
    current_df_grouped['Hour'] = current_df_grouped.index.hour

    neighborhood_dict_grouped[label] = current_df_grouped


In [175]:
neighborhood_dict_grouped_diff_week = {}

# Loop through the original nested_dict
for label in neighborhood_dict_grouped:
    # Extract the DataFrame for the current label, month, and hour
    current_df = neighborhood_dict_grouped[label]
    
    current_df['Date'] = pd.to_datetime(current_df_grouped["Date"])
        # Apply the transformations
    current_df["Weekday"] = current_df["Date"].dt.weekday
    current_df["Month"] = current_df["Date"].dt.month
    current_df['IsWeekend'] = current_df['Weekday'].apply(lambda day: 1 if day in ['Saturday', 'Sunday'] else 0)

    neighborhood_dict_grouped[label] = current_df

In [176]:
for label in neighborhood_dict_grouped:
    neighborhood_dict_grouped[label] =  neighborhood_dict_grouped[label].rename(columns={'Count': f'Count_{label}'})

In [177]:
# checking if all have the same lenght

true_list = []
for label in neighborhood_dict_grouped:
    if len(neighborhood_dict_grouped[label]) == 4391:
        true_list.append(True)
    else:
        true_list.append(False)

if all(true_list):
    print("All elements in the list are True")

All elements in the list are True


In [178]:
for label in neighborhood_dict_grouped:
    if label > 0:
        neighborhood_dict_grouped[0][f'Count_{label}'] = neighborhood_dict_grouped[label][f'Count_{label}']

df_all_counts = neighborhood_dict_grouped[0]

In [179]:
notebook_path = os.path.abspath("1. Data_Preparation.ipynb")
notebook_directory = os.path.dirname(notebook_path)

replacement_string = "data\\interim\\Cluster_Data_All_Month\\"
index = notebook_directory.find("\\notebooks")
modified_path = notebook_directory[:index]

modified_path += "\\" + replacement_string

In [180]:
file_name = f"df_all_counts_grouped.csv"

# Create the full paths for saving
full_path = modified_path + file_name

df_all_counts.to_csv(full_path, index=True)
